In [1]:
import os

In [ ]:
!zip -r 20230119_anamika_works_nlp.zip /content/drive/MyDrive/Colab_Notebooks

  adding: content/drive/MyDrive/Colab_Notebooks/ (stored 0%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/ (stored 0%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/Cheque100828.jpg (deflated 26%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/Cheque120613.jpg (deflated 30%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/Cheque120620.jpg (deflated 30%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/Cheque309069.jpg (deflated 36%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/Cheque100828.json (deflated 69%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/Cheque120613.json (deflated 68%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/Cheque120620.json (deflated 69%)
  adding: content/drive/MyDrive/Colab_Notebooks/sample_data/Cheque309069.json (deflated 67%)
  adding: content/drive/MyDrive/Colab_Notebooks/2023-01-19 06:06 model Epoch 10 Total-epoch51/ (stored 0%)
  adding: content/d

In [2]:
# !pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

In [3]:
# import PIL
# !pip install pillow==7.0.0

In [4]:
import glob
import json
import itertools


from pdf2image import convert_from_path


import pandas as pd
import numpy as np
from PIL import Image

# Huggingface
from datasets import Dataset

In [5]:
from transformers import LayoutLMv2Tokenizer, LayoutLMv2FeatureExtractor

In [6]:
from transformers import AutoProcessor, AutoModelForQuestionAnswering
import torch

processor = AutoProcessor.from_pretrained("tiennvcs/layoutlmv2-base-uncased-finetuned-docvqa", apply_ocr=False)
model = AutoModelForQuestionAnswering.from_pretrained("tiennvcs/layoutlmv2-base-uncased-finetuned-docvqa" )

In [7]:
base_model_checkpoint = "tiennvcs/layoutlmv2-base-uncased-finetuned-docvqa"

model_image_size = (224, 224)

datapoint_names = [s.strip() for s in "bankname, amount, date, accountno".split(",")]

In [8]:
tokenizer = LayoutLMv2Tokenizer.from_pretrained(base_model_checkpoint)
feature_extractor = LayoutLMv2FeatureExtractor()

In [40]:

def get_ocr_words_and_boxes(examples): 
    
    # get a batch of document images
    images = [Image.open(os.path.join('/content/',images)).convert("RGB") for images in examples['pdffile']]
    
    # resize every image to 224x224 + apply tesseract to get words + normalized boxes
    encoded_inputs = feature_extractor(images)

    examples['image'] = encoded_inputs.pixel_values
    examples['words'] = encoded_inputs.words
    examples['boxes'] = encoded_inputs.boxes

    return examples

In [9]:
def dataset_from_folder(foldername='/content/'):
    data = []
    count = 0

    for images in glob.glob(f"{foldername}/*.jpg"):
      annotation_file = images.split('.')[0]+'.json'
      if os.path.exists(annotation_file):
          count = count + 1
          with open(annotation_file) as fp:
              annotations = json.load(fp)
      
          dp = []
          annotated_datapoints =[]
          #print("annotation file", annotation_file)
          annotated_datapoints = [annotation_dp["datapoint_name"].strip() for annotation_dp in annotations['datapoints'] ]
  #                     print(annotated_datapoints)
          for datapoint_name in datapoint_names:
              found = False
              question = f"What is the {datapoint_name}?"
              answer = ""
              answer_bbox = []
              
              for annotation in annotations['datapoints']:                            
                  if annotation["datapoint_name"].strip() == datapoint_name:
                      if len(annotation["bbox"]) == 4:
                          found = True
                          # if annotation["corrected_text"] == "":
                          answer = annotation["annotatedtext"]
                          # else:
                          #     answer = annotation["corrected_text"]
                          answer_bbox = annotation["bbox"]
                      elif len(annotation["bbox"]) == 0:
                          found = True
                          answer = "Empty"
                          answer_bbox = [0,0,1,1]
                  
                      
              if found:
                  data.append([pdffile, annotation_file, question, answer, answer_bbox])
                  
    df = pd.DataFrame(data, columns=["images", "annotationfile", "question", "answer", "answer_bbox"])
    dataset = Dataset.from_pandas(df)
    print(count)
    return dataset




In [10]:
import torch
torch.cuda.is_available()

False

In [11]:
# !pip install pdfinfo
# !apt-get install -y poppler-utils

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !sudo apt-get install tesseract-ocr
# !pip install pytesseract

In [17]:
import pytesseract
# Load images and json files
dataset_train = dataset_from_folder("/content/drive/MyDrive/sample_data/")
# Run OCR and resize the image
dataset_train = dataset_train.map(get_ocr_words_and_boxes, batched=True, batch_size=5)

4


  0%|          | 0/4 [00:00<?, ?ba/s]

In [14]:
# pick a random example
# example = data['datapoints'][1]
# for k,v in example.items():
#   print(k + ":", v)
# import pandas as pd

# df = pd.DataFrame(data['datapoints'])
# df.head()

,datapoint_name,imagefile,annotatedtext,bbox
0,bankname,Cheque100828.jpg,ICICI Bank,"[197, 72, 480, 110]"
1,amount,Cheque100828.jpg,2223000,"[1890, 415, 2213, 485]"
2,date,Cheque100828.jpg,25012016,"[1835, 86, 2318, 148]"
3,accountno,Cheque100828.jpg,63080155452,"[405, 558, 638, 604]"


In [ ]:
dataset_train.to_pandas().head()

In [19]:
from PIL import Image
from datasets import Dataset, Features, Sequence, Value, Array2D, Array3D
import torch
import torchvision.ops.boxes as torch_boxes

In [20]:
def find_overlapping_boxes(search_bbox, bboxes, ratio=0.0):
    found_boxes = []
    for j, bbox in enumerate(bboxes):
        iou = torch_boxes.box_iou(torch.tensor([search_bbox], dtype=torch.float), torch.tensor([bbox], dtype=torch.float))
        if iou > ratio:
            found_boxes.append(j)
    return found_boxes

In [21]:
def encode_dataset(examples, max_length=512):    
    questions = examples['question']
    words = examples['words']
    boxes = examples['boxes']
    

    
    start_positions = []
    end_positions = []
    
    encoding = tokenizer(questions, words, boxes, max_length=max_length, padding="max_length", truncation=True)
    
    for i in range(len(examples['images'])):
        search_bbox = examples["answer_bbox"]
        # print(search_bbox)
        overlaps = find_overlapping_boxes(search_bbox[i], encoding['bbox'][i], ratio=0.0)
        if len(overlaps) > 0:
            # print(overlaps[0])
            start_positions.append(overlaps[0])
            end_positions.append(overlaps[-1])
        else:
            # We havent found any overlaps. Maybe because OCR didn't detect a word here, or because the token was truncated?
            print("skip this one...")
            start_positions.append(0)
            end_positions.append(0)
    
    encoding['image'] = examples['image']
    #encoding['gold_answer'] = examples['answer']
    encoding['start_positions'] = start_positions
    encoding['end_positions'] = end_positions
    
    return encoding

def filter_dataset(example):
    if example['start_positions'] == 0 and example['end_positions'] == 0:
        return False
    else:
        return True

In [ ]:
# we need to define custom features
features = Features({
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'start_positions': Value(dtype='int64'),
    'end_positions': Value(dtype='int64'),
})

# TRAIN
encoded_dataset_train = dataset_train.map(encode_dataset, batched=True, batch_size=5, 
                                remove_columns=dataset_train.column_names,
                                features=features)

encoded_dataset_train = encoded_dataset_train.filter(filter_dataset)

# VALIDATION

encoded_dataset_val = dataset_train.map(encode_dataset, batched=True, batch_size=5, 
                                remove_columns=dataset_train.column_names,
                                features=features)

encoded_dataset_val = encoded_dataset_train.filter(filter_dataset)
# print(encoded_dataset_val)

In [23]:
encoded_dataset_train.set_format(type="torch")
dataloader_train = torch.utils.data.DataLoader(encoded_dataset_train, batch_size=4)

encoded_dataset_val.set_format(type="torch")
dataloader_val = torch.utils.data.DataLoader(encoded_dataset_val, batch_size=4)

In [24]:
current_epoch = 0
losses = []

In [26]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(base_model_checkpoint)

from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()
for epoch in range(50):  # loop over the dataset multiple times
    for idx, batch in enumerate(dataloader_train):
        # get the inputs;
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        bbox = batch["bbox"].to(device)
        image = batch["image"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,token_type_ids = token_type_ids,
                        bbox=bbox, image=image, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        losses.append((current_epoch, idx, loss.item()))
        print("(Epoch: {epoch} Loss:, {loss.item()}")

    model.eval()
    
print('training completed!')

..


In [27]:
!git clone https://huggingface.co/microsoft/layoutlmv2-base-uncased

Cloning into 'layoutlmv2-base-uncased'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 51 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [25]:
from transformers import LayoutLMv2Processor


os.environ["TOKENIZERS_PARALLELISM"] = "false"
processor = LayoutLMv2Processor.from_pretrained("/content/layoutlmv2-base-uncased")

def predict(m, image, question,words,bboxes, putOnDevice=True):
    encoding = processor(
        images=image,
        text=question,
        text_pair=words,
        boxes=bboxes,
        return_tensors="pt",
        truncation=True,
    )
    
    if putOnDevice:
        for k,v in encoding.items():
            encoding[k] = v.to(m.device)
    outputs = m(**encoding)

    # step 3: get start_logits and end_logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # step 4: get largest logit for both
    predicted_start_idx = start_logits.argmax(-1).item()
    predicted_end_idx = end_logits.argmax(-1).item()

    # step 5: decode the predicted answer
    answer = processor.tokenizer.decode(encoding.input_ids.squeeze()[predicted_start_idx:predicted_end_idx+1])
    return answer



In [26]:
from transformers import (
    LayoutLMv2FeatureExtractor,
    LayoutLMv2ForQuestionAnswering,
    LayoutLMv2Processor,
    LayoutLMv2Tokenizer,
    pipeline,
)
modelpath = "/content/2023-01-19 06:06 model Epoch 10 Total-epoch51/"
model = LayoutLMv2ForQuestionAnswering.from_pretrained(modelpath,ignore_mismatched_sizes=True)

In [34]:
# !sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.14 [154 kB]
Fetched 154 kB in 0s (455 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend

In [ ]:
# import json

# with open('/content/Cheque100828.json') as f:
#   data = json.load(f)

In [37]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# example = data['datapoints'][0]
# root_dir = '/content/'
# question = example['question']
# image = Image.open(root_dir + example['image']).convert("RGB")
# print(question)
# image

In [38]:
%%time

from transformers import LayoutLMv2Processor, LayoutLMv2ForQuestionAnswering, set_seed
import torch
from PIL import Image
# from datasets import load_dataset
from transformers import LayoutLMv2Config

set_seed(88)
processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased")
model = LayoutLMv2ForQuestionAnswering.from_pretrained("/content/2023-01-19 06:06 model Epoch 10 Total-epoch51/")

image_path = "/content/drive/MyDrive/sample_data/Cheque309069.jpg"
image = Image.open(image_path).convert("RGB")
question = "accountno?"
encoding = processor(image, question, return_tensors="pt")

outputs = model(**encoding)
predicted_start_idx = outputs.start_logits.argmax(-1).item()
predicted_end_idx = outputs.end_logits.argmax(-1).item()
predicted_start_idx, predicted_end_idx

predicted_answer_tokens = encoding.input_ids.squeeze()[predicted_start_idx : predicted_end_idx + 1]
predicted_answer = processor.tokenizer.decode(predicted_answer_tokens)

CPU times: user 8.64 s, sys: 825 ms, total: 9.46 s
Wall time: 12.8 s


In [39]:
predicted_answer

'911010049001545'